In [2]:
from algbench import describe, read_as_pandas

In [3]:
describe("./results")

An entry in the database can look like this:
_____________________________________________
 result:
| ub: 473.3792386060225
| lb: 441.44809426510574
| stats:
|| num_explored: 13954
|| num_branches: 13951
|| num_iterations: 15954
| n: 126
 timestamp: 2023-05-26T15:23:10.806535
 runtime: 120.2784583568573
 stdout: 
 stderr: 
 env_fingerprint: dff294d2e80a73937f84e9442a654782dd0bc7f4
 args_fingerprint: 665eafaacacc672b1ea4702fcccdb745b1ed97b3
 parameters:
| func: measure
| args:
|| instance_name: rnd_ol01/CETSP-200-0
|| configuration:
||| root: ConvexHull
||| branching: ChFarthestCircleSimplifying
||| search: DfsBfs
||| rules: []
||| num_threads: 8
|| timelimit: 120
 argv: ['/ibr/home/krupke/anaconda3/envs/mo310/lib/python3.10/site-packages/slurmina...
 env:
| hostname: algry01
| python_version: 3.10.8 (main, Nov 24 2022, 14:13:03) [GCC 11.2.0]
| python: /ibr/home/krupke/anaconda3/envs/mo310/bin/python3
| cwd: /misc/ibr/home/krupke/close-enough-tsp/benchmark
| environment: [{'name': 'tzda

In [69]:
result = read_as_pandas("./results/", lambda data: {"instance": data["parameters"]["args"]["instance_name"],
                                                    "alg": str(data["parameters"]["args"]["configuration"]),
                                                    "conf": data["parameters"]["args"]["configuration"],
                                                    "lb": data["result"]["lb"],
                                                    "ub": data["result"]["ub"],
                                                    "runtime": data["runtime"],
                                                    "iterations": float(data["result"]["stats"]["num_iterations"]),
                                                    "n": data["result"]["n"]})
result["gap"] = (1-result["lb"]/result["ub"])*100
result["gap_above_tol"] = result["gap"].apply((lambda x: max(0, x-1.0)))
result.drop_duplicates(subset=["alg", "instance"], inplace=True)
result = result[result["n"]>=80]
result.groupby("alg")[["runtime", "gap", "gap_above_tol", "iterations"]].mean()

,runtime,gap,gap_above_tol,iterations
alg,,,,
"{'root': 'ConvexHull', 'branching': 'ChFarthestCircle', 'search': 'DfsBfs', 'rules': [], 'num_threads': 8}",120.219121,3.468589,2.468589,14265.2
"{'root': 'ConvexHull', 'branching': 'ChFarthestCircleSimplifying', 'search': 'CheapestBreadthFirst', 'rules': [], 'num_threads': 8}",120.085117,20.940972,19.940972,5959.4
"{'root': 'ConvexHull', 'branching': 'ChFarthestCircleSimplifying', 'search': 'CheapestChildDepthFirst', 'rules': [], 'num_threads': 8}",120.176760,23.702639,22.702639,290477.6
"{'root': 'ConvexHull', 'branching': 'ChFarthestCircleSimplifying', 'search': 'DfsBfs', 'rules': [], 'num_threads': 8}",120.244555,3.179265,2.179265,18301.0
"{'root': 'LongestEdgePlusFurthestCircle', 'branching': 'ChFarthestCircleSimplifying', 'search': 'CheapestBreadthFirst', 'rules': [], 'num_threads': 8}",120.080071,21.708667,20.708667,7405.0
"{'root': 'LongestEdgePlusFurthestCircle', 'branching': 'FarthestCircle', 'search': 'CheapestBreadthFirst', 'rules': [], 'num_threads': 8}",120.096687,20.154550,19.154550,7179.4
"{'root': 'Random', 'branching': 'Random', 'search': 'Random', 'rules': [], 'num_threads': 8}",120.318086,62.302829,61.302829,67255.4


In [70]:
result["instance"].nunique()

5

In [65]:
result[result["ub"]<=1.02*result["lb"]].groupby(["alg"])[["instance"]].nunique()

,instance
alg,


In [6]:
import pandas as pd
t = pd.merge(result.groupby(["instance"])[["lb"]].max(), result.groupby(["instance"])[["ub"]].min(), on="instance")
(t["lb"]<=t["ub"]+0.001).all()

True

In [7]:
t = pd.merge(result.groupby(["instance"])[["lb"]].min(), result.groupby(["instance"])[["ub"]].max(), on="instance")
t[t["ub"]<=t["lb"]*1.05].index.unique().to_list()
#t

['rnd_ol01/CETSP-30-0',
 'rnd_ol01/CETSP-30-2',
 'rnd_ol01/CETSP-30-4',
 'rnd_ol01/CETSP-40-2',
 'rnd_ol01/CETSP-40-4']

In [47]:
import random
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option("display.max_colwidth", 500)
t = result[result["instance"]==random.choice(result["instance"].unique().tolist())].sort_values(by=["alg"]).drop(columns="conf")
t

,instance,alg,lb,ub,runtime,iterations,gap,gap_above_tol
15,rnd_ol01/CETSP-200-2,"{'root': 'ConvexHull', 'branching': 'ChFarthestCircle', 'search': 'DfsBfs', 'rules': [], 'num_threads': 8}",423.751357,434.447965,120.210610,13524.0,2.462115,1.462115
19,rnd_ol01/CETSP-200-2,"{'root': 'ConvexHull', 'branching': 'ChFarthestCircleSimplifying', 'search': 'CheapestBreadthFirst', 'rules': [], 'num_threads': 8}",423.407150,522.454406,120.089697,6066.0,18.958067,17.958067
18,rnd_ol01/CETSP-200-2,"{'root': 'ConvexHull', 'branching': 'ChFarthestCircleSimplifying', 'search': 'CheapestChildDepthFirst', 'rules': [], 'num_threads': 8}",347.306387,435.458376,120.182193,302382.0,20.243494,19.243494
14,rnd_ol01/CETSP-200-2,"{'root': 'ConvexHull', 'branching': 'ChFarthestCircleSimplifying', 'search': 'DfsBfs', 'rules': [], 'num_threads': 8}",424.812119,434.448005,120.239580,17918.0,2.217961,1.217961
16,rnd_ol01/CETSP-200-2,"{'root': 'LongestEdgePlusFurthestCircle', 'branching': 'ChFarthestCircleSimplifying', 'search': 'CheapestBreadthFirst', 'rules': [], 'num_threads': 8}",414.133060,543.995324,120.085940,7632.0,23.871945,22.871945
17,rnd_ol01/CETSP-200-2,"{'root': 'LongestEdgePlusFurthestCircle', 'branching': 'FarthestCircle', 'search': 'CheapestBreadthFirst', 'rules': [], 'num_threads': 8}",413.483749,511.733905,120.103549,7296.0,19.199462,18.199462
20,rnd_ol01/CETSP-200-2,"{'root': 'Random', 'branching': 'Random', 'search': 'Random', 'rules': [], 'num_threads': 8}",191.693957,557.393296,120.317755,57898.0,65.608851,64.608851
